In [1]:
# Esmalt installi ja aktiveeri:
#conda install -c huggingface transformers
#conda install pytorch torchvision torchaudio cudatoolkit=10.2 -c pytorch
#conda create -n tf tensorflow
#conda activate tf

In [ ]:
#https://github.com/estnltk/estnltk/blob/1.6.7b_with_hfst_python_dependency/tutorials/taggers/embeddings_tagger.ipynb
#https://github.com/huggingface/transformers
#https://pytorch.org/
#https://www.tensorflow.org/
#https://github.com/google-research/bert/blob/a9ba4b8d7704c1ae18d1b28c56c0430d41407eb1/multilingual.md

In [1]:
from transformers import AutoTokenizer, AutoModelForMaskedLM

In [2]:
tokenizer = AutoTokenizer.from_pretrained("tartuNLP/EstBERT")
model = AutoModelForMaskedLM.from_pretrained("tartuNLP/EstBERT")

In [3]:
import torch
from typing import MutableMapping, List
from transformers import BertTokenizer, logging, BertModel

logging.set_verbosity(30)
from estnltk.text import Text
from estnltk.taggers import Tagger
from estnltk.layer.layer import Layer
import numpy as np


class BertTagger(Tagger):
    """Tags BERT embeddings."""

    def __init__(self, bert_location: str, sentences_layer: str = 'sentences',
                 token_level: bool = False,
                 output_layer: str = 'bert_embeddings', bert_layers: List[int] = None, method='concatenate'):

        if bert_layers is None:
            bert_layers = [-4, -3, -2, -1]
        else:
            for layer in bert_layers:
                if abs(layer) > 12:
                    msg = "BERT base model only has 12 layers of transformer encoder, chose layers from (-12..-1). It " \
                          "is reasonable to choose layers from the last layers, for example [-4, -3, -2, -1]: last 4 " \
                          "layers. "
                    raise Exception(msg)
        self.conf_param = ('bert_location', 'bert_model', 'tokenizer', 'method', 'token_level', 'bert_layers')
        if bert_location is None:
            msg = "Directory containing BERT model must be specified."
            raise Exception(msg)
        else:
            self.bert_location = bert_location
        if method not in ('concatenate', 'add', 'all'):
            msg = "Method can be 'concatenate', 'add' or 'all'."
            raise Exception(msg)
        self.method = method
        self.output_layer = output_layer
        self.input_layers = [sentences_layer]

        self.bert_model = BertModel.from_pretrained(bert_location, output_hidden_states=True)

        self.tokenizer = BertTokenizer.from_pretrained(self.bert_location)

        self.output_attributes = ['token', 'bert_embedding']

        self.token_level = token_level
        self.bert_layers = bert_layers

    def _make_layer(self, text: Text, layers: MutableMapping[str, Layer], status: dict) -> Layer:
        sentences_layer = layers[self.input_layers[0]]
        embeddings_layer = Layer(name=self.output_layer, text_object=text, attributes=self.output_attributes,
                                 ambiguous=True)

        start, i = 0, 0
        word_spans = []

        for k, sentence in enumerate(sentences_layer):

            for word in sentence:
                word_spans.append((word.start, word.end, word.text))
            sent_text = sentence.enclosing_text

            embeddings = get_embeddings(sent_text, self.bert_model, self.tokenizer, self.method, self.bert_layers)[
                         1:-1]  # first one in start token, and last one is sep token
            tokens = self.tokenizer.tokenize(sent_text)
            assert len(tokens) == len(embeddings)
            if k != 0:  # move the start manually when next sentence starts
                start = word_spans[i][0]

            if self.token_level:  # annotates tokens
                for j, packed in enumerate(zip(embeddings, tokens)):
                    token_emb, token_init = packed[0], packed[1]

                    if not token_init.startswith("#") and j != 0:  # move to next word
                        if start == word_spans[i][1]:  # BERT's wordpiece tokenizer can tokenize differently
                            i += 1  # next word starts
                            word_span = word_spans[i]
                            start = word_span[0]  # the start id of this word
                    if self.method == 'all':
                        embedding = []
                        for tok_emb in token_emb:
                            emb = []
                            for e in tok_emb:
                                emb.append(float(e))
                            embedding.append(emb)
                    else:
                        embedding = [float(t) for t in token_emb]

                    attributes = {'token': token_init, 'bert_embedding': embedding}
                    token = token_init.strip()
                    embeddings_layer.add_annotation((start, start + len(token.replace('#', ''))), **attributes)
                    start += len(token.replace('#', ''))  # adding token length to the current pointer

                i += 1  # move the pointer manually

            else:  # annotates full words, adding the token level embedding together
                collected_tokens = []
                collected_embeddings = []
                for j, packed in enumerate(zip(embeddings, tokens)):
                    token_emb, token_init = packed[0], packed[1]

                    if i == 0 and j == 0:
                        start = 0
                        collected_embeddings.append(token_emb)
                        collected_tokens.append(token_init)
                        start += len(token_init)

                    if i != 0 and j == 0:
                        collected_tokens = [token_init]
                        collected_embeddings = [token_emb]
                        start = word_spans[i][0]

                        start += len(token_init)

                    if not token_init.startswith("#") and j != 0:  # move to next word
                        if start == word_spans[i][1]:  # BERT's wordpiece tokenizer can tokenize differently

                            if collected_embeddings:
                                if self.method == 'all':
                                    embedding = []
                                    for tok_embs in collected_embeddings:
                                        token_embs = []
                                        for embs in tok_embs:
                                            token_embs_emb = []
                                            for emb in embs:
                                                token_embs_emb.append(float(emb))
                                            token_embs.append(token_embs_emb)
                                        embedding.append(token_embs)
                                else:
                                    embedding = [float(t) for t in np.sum(collected_embeddings, 0)]

                                attributes = {'token': collected_tokens, 'bert_embedding': embedding}
                                embeddings_layer.add_annotation((word_spans[i][0], word_spans[i][1]),
                                                                **attributes)

                            i += 1  # next word starts
                            start = word_spans[i][0]  # the start id of this word
                            collected_embeddings = [token_emb]
                            collected_tokens = [token_init]
                            start += len(token_init)
                        else:
                            start += len(token_init)
                            collected_tokens.append(token_init)
                            collected_embeddings.append(token_emb)
                    elif token_init.startswith("#"):
                        collected_tokens.append(token_init)
                        collected_embeddings.append(token_emb)
                        start += len(token_init.replace("#", ''))

                if collected_tokens:
                    if self.method == 'all':
                        embedding = []
                        for tok_embs in collected_embeddings:
                            token_embs = []
                            for embs in tok_embs:
                                token_embs_emb = []
                                for emb in embs:
                                    token_embs_emb.append(float(emb))
                                token_embs.append(token_embs_emb)
                            embedding.append(token_embs)
                    else:
                        embedding = [float(t) for t in np.sum(collected_embeddings, 0)]

                    attributes = {'token': collected_tokens, 'bert_embedding': embedding}
                    embeddings_layer.add_annotation((word_spans[i][0], word_spans[i][1]),
                                                    **attributes)

                i += 1  # move the pointer manually

        return embeddings_layer


def get_embeddings(sentence: str, model, tokenizer, method, bert_layers):
    input_data = tokenizer.encode_plus(sentence)
    input_ids = input_data.get('input_ids')
    token_vecs_cat = []
    if len(input_ids) > 512:  # maximum sequence length can be 512
        msg = "Input sentence is too big (%s), splitting the sentence." % len(input_ids)
        print(msg)
        collected_input_ids = []
        while True:
            collected_input_ids.append(input_ids[:512])
            input_ids = input_ids[512:]
            if len(input_ids) <= 512:
                collected_input_ids.append(input_ids)
                break
    else:
        collected_input_ids = [input_ids]

    for i, input_ids in enumerate(collected_input_ids):

        segments_ids = [1] * len(input_ids)
        tokens_tensor = torch.tensor([input_ids])
        segments_tensors = torch.tensor([segments_ids])

        with torch.no_grad():
            outputs = model(tokens_tensor, segments_tensors)
            hidden_states = outputs[2]
        token_embeddings = torch.stack(hidden_states, dim=0)
        token_embeddings = torch.squeeze(token_embeddings, dim=1)
        token_embeddings = token_embeddings.permute(1, 0, 2)

        for token in token_embeddings:
            if method == 'concatenate':  # concatenate the vectors
                layers = [token[i] for i in bert_layers]
                cat_vec = torch.cat(layers, dim=0)
                token_vecs_cat.append(np.asarray(cat_vec))

            if method == 'add':  # elementwise addition
                layers = [np.asarray(token[i]) for i in bert_layers]
                sum_vec = np.sum(layers, 0)
                token_vecs_cat.append(np.asarray(sum_vec))

            if method == 'all':  # return all
                layers = [np.asarray(token[i]) for i in bert_layers]
                token_vecs_cat.append(layers)

    return token_vecs_cat

In [4]:
bert_tagger = BertTagger(bert_location='tartuNLP/EstBERT')

Some weights of BertModel were not initialized from the model checkpoint at tartuNLP/EstBERT and are newly initialized: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
import pandas as pd
data = pd.read_csv("Koondtabel_lemmadega.csv")
data.head()

,Tag,Lõigu sisu,Pealkiri1,Kohtuasja_nr,Kohtuasja_ID,algus_lopp,stop_eemaldatud,Lemma
0,C01PointnumeroteAltN,"27 Kõigepealt väärib märkimist, et väljakujunenud kohtupraktika kohaselt on ELTL artiklis 267 sätestatud liikmesriigi kohtute ja Euroopa Kohtu vahelises koostöömenetluses Euroopa Kohtu ülesanne anda liikmesriigi kohtule tarvilik vastus, mis võimaldaks viimasel poolelioleva kohtuasja lahendada. Seda arvestades tuleb Euroopa Kohtul temale esitatud küsimused vajaduse korral ümber sõnastada (2. aprilli 2020. aasta kohtuotsus Ruska Federacija, C‑897/19 PPU, EU:C:2020:262, punkt 43 ja seal viidatud kohtupraktika).",NaN,C‑416/20 PPU,0,algus,27 väärib märkimist väljakujunenud kohtupraktika eltl artiklis 267 sätestatud liikmesriigi kohtute euroopa kohtu vahelises koostöömenetluses euroopa kohtu ülesanne anda liikmesriigi kohtule tarvilik vastus võimaldaks viimasel poolelioleva kohtuasja lahendada arvestades euroopa kohtul esitatud küsimused vajaduse sõnastada 2. aprilli 2020. aasta kohtuotsus ruska federacija c‑897/19 ppu eu c:2020:262 punkt 43 viidatud kohtupraktika,27 väärima märkimine väljakujunenud kohtupraktika eltl artikkel 267 sätestama liikmesriik kohus euroopa kohus vaheline koostöömenetlus euroopa kohus ülesanne andma liikmesriik kohus tarvilik vastus võimaldama viimane pooleliolev kohtuasi lahendama arvestama euroopa kohus esitama küsimus vajadus sõnastama 2. aprill 2020. aasta kohtuotsus ruskama federacija c ‑ 897 / 19 ppu eu c : 2020 : 262 punkt 43 viidatud kohtupraktika
1,C01PointnumeroteAltN,"28 Käesoleval juhul nähtub eelotsusetaotlusest, et eelotsusetaotluse esitanud kohus peab tegema otsuse selle kohta, kas TRi üleandmine Rumeenia ametiasutustele on õiguspärane seaduse rahvusvahelise õigusabi kohta kriminaalasjades § 83 sätete alusel, millega rakendatakse raamotsuse 2002/584 artiklit 4a Saksa õiguses.",NaN,C‑416/20 PPU,0,algus,28 käesoleval nähtub eelotsusetaotlusest eelotsusetaotluse esitanud kohus otsuse tri üleandmine rumeenia ametiasutustele õiguspärane seaduse rahvusvahelise õigusabi kriminaalasjades § 83 sätete alusel rakendatakse raamotsuse 2002/584 artiklit 4a saksa õiguses,28 käesolev nähtuma eelotsusetaotlus eelotsusetaotlus esitama kohus otsus tri üleandmine rumeenia ametiasutus õiguspärane seadus rahvusvaheline õigusabi kriminaalasi § 83 säte alune rakendama raamotsus 2002 / 584 artikkel 4 saksa õigus
2,C01PointnumeroteAltN,"29 Eelotsusetaotluse esitanud kohtu hinnangul on üleandmiseks vajalikud tingimused täidetud, kuna esiteks on see isik teadlikult hoidunud menetlustest – mille tulemusel on tehtud need Euroopa vahistamismäärused, mida see kohus peab täitma –, põgenedes Saksamaale ja takistades seega talle kohtukutse isiklikult kättetoimetamist, ning teiseks esindas seda isikut esimeses kohtuastmes tema valitud advokaat ja apellatsiooniastmes kohtu määratud kaitsja. TR väitis aga selles kohtus, et see üleandmine ei ole direktiivi 2016/343 artiklites 8 ja 9 sätestatud nõudeid arvestades õiguspärane, kuna Rumeenias ei ole mingit tagatist tema suhtes toimunud kriminaalmenetluse uuendamiseks.",NaN,C‑416/20 PPU,0,algus,29 eelotsusetaotluse esitanud kohtu hinnangul üleandmiseks vajalikud tingimused täidetud esiteks isik teadlikult hoidunud menetlustest – tulemusel euroopa vahistamismäärused kohus täitma – põgenedes saksamaale takistades kohtukutse isiklikult kättetoimetamist esindas isikut esimeses kohtuastmes valitud advokaat apellatsiooniastmes kohtu määratud kaitsja tr väitis kohtus üleandmine direktiivi 2016/343 artiklites 8 9 sätestatud nõudeid arvestades õiguspärane rumeenias tagatist toimunud kriminaalmenetluse uuendamiseks,29 eelotsusetaotlus esitama kohus hinnang üleandmine vajalik tingimus täidetud esiteks isik teadlikult hoiduma menetlus - tulemus euroopa vahistamismäärus kohus täitma - põgenema saksamaa takistama kohtukutse isiklikult kättetoimetamine esindama isik esimene kohtuaste valima advokaat apellatsiooniaste kohus määrama kaitsja tr väitma kohus üleandmine direktiiv 2016 / 343 artikk

In [7]:
column_list=data['Lemma'].to_list()
column_list[0:100]

['27 väärima märkimine väljakujunenud kohtupraktika eltl artikkel 267 sätestama liikmesriik kohus euroopa kohus vaheline koostöömenetlus euroopa kohus ülesanne andma liikmesriik kohus tarvilik vastus võimaldama viimane pooleliolev kohtuasi lahendama arvestama euroopa kohus esitama küsimus vajadus sõnastama 2. aprill 2020. aasta kohtuotsus ruskama federacija c ‑ 897 / 19 ppu eu c : 2020 : 262 punkt 43 viidatud kohtupraktika',
 '28 käesolev nähtuma eelotsusetaotlus eelotsusetaotlus esitama kohus otsus tri üleandmine rumeenia ametiasutus õiguspärane seadus rahvusvaheline õigusabi kriminaalasi § 83 säte alune rakendama raamotsus 2002 / 584 artikkel 4 saksa õigus',
 '29 eelotsusetaotlus esitama kohus hinnang üleandmine vajalik tingimus täidetud esiteks isik teadlikult hoiduma menetlus - tulemus euroopa vahistamismäärus kohus täitma - põgenema saksamaa takistama kohtukutse isiklikult kättetoimetamine esindama isik esimene kohtuaste valima advokaat apellatsiooniaste kohus määrama kaitsja tr v

In [9]:
# koondab iga sõna vektorid üheks ning lisab need ühte listi
row_nr=[]
vektor=[]
i=0
for i in range(len(column_list)):
    text = Text(column_list[i])
    text.analyse('segmentation')
    bert_tagger.tag(text)
    frame = text.bert_embeddings
    listike = np.sum(frame['bert_embedding'], axis=0)
    row_nr.append(i)
    vektor.append(listike[0])
    if i%1000==0:
        print(i)


0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
Input sentence is too big (524), splitting the sentence.
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000

In [11]:
#Salvestan andmed
np.save('BERT_embeddings.npy', vektor)

In [19]:
#Kasutamiseks
all_embeddings = np.load('BERT_embeddings.npy')

In [ ]:
#testandmed
test_text = Text("sisendkäibemaksu mahaarvamine")
test_text.analyse('segmentation')
bert_tagger.tag(test_text)
frame = test_text.bert_embeddings
proovivektoriks1=np.sum(frame["bert_embedding"], axis=0)

In [ ]:
#Salvestan testandmed
np.save('proovivektor.npy', proovivektoriks1)

In [20]:
#Kasutamiseks
proovivektoriks1 = np.load('proovivektor.npy')

In [ ]:
#Kuivõrd BERT_embeddings.npy fail oli liiga suur jupyteris (arvuti mälu oli liiga väike) 
#ja colabis koosiinuse võrdluste tegemiseks,
#siis järgmised koodiosad jooksutati läbi eraldi serveris, seetõttu oli vajalik ka andmete vahesalvestamine

In [ ]:
# Esmalt oli vaja kätte saada võrdlus kõigi andmetega

In [ ]:
import numpy as np

In [ ]:
all_embeddings = np.load('BERT_embeddings.npy')

In [ ]:
proovivektoriks1 = np.load('proovivektor.npy')

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
data=pd.read_csv("Koondtabel_lemmadega.csv")

In [ ]:
#Loon tabeli, kus on koossiinuse sarnaseimad 200 indeksit, sarnasus ja järjekord (kui sarnane on)
sarnased_maatriks=cosine_similarity(all_embeddings, proovivektoriks1.reshape(1, -1))
sorteeritud_indeksid=np.argsort(np.squeeze(sarnased_maatriks))
listik_indx=[]
for i in range(1,201):
  listik_indx.append(sorteeritud_indeksid[len(sorteeritud_indeksid) - i])
koos_dist=sarnased_maatriks[listik_indx]
koos_dist_jarj=[]
for i in koos_dist:
    koos_dist_jarj.append(i[0]) 
print(koos_dist_jarj)
print(listik_indx)
tabel_sarnasus = pd.DataFrame(columns=['Kohtuasja_indx', 'Koos_sarnasus', 'Jarjekord'])
tabel_sarnasus['Kohtuasja_indx'] = listik_indx
tabel_sarnasus['Koos_sarnasus'] = koos_dist_jarj
tabel_sarnasus['Jarjekord'] = list(range(1, 201))
Tabel_sarnased=tabel_sarnasus.set_index('Kohtuasja_indx') 
Tabel_sarnased

In [ ]:
#Filtreerin indeksite põhjal välja sarnased lõigud
filtreeritud_sarnased=data[data.index.isin(listik_indx)]
filtreeritud_sarnased

In [ ]:
# Ühendan kaks tabelit ja filtreerin välja Koos_sarnasus,	Jarjekord, Lõigu sisu, Kohtuasja_nr
Tulemus = pd.concat([Tabel_sarnased, filtreeritud_sarnased], axis=1)
uus = (Tulemus[['Koos_sarnasus', 'Jarjekord','Tag','Lõigu sisu', 'Pealkiri1','Kohtuasja_nr','Kohtuasja_ID', 'algus_lopp', 'Lemma']]).sort_values('Jarjekord', ascending=True)
uus['Indeks'] = uus.index
uus

In [ ]:
#Salvestan tulemuse
uus.to_csv("bert_kogu_tulemus.csv", index=False)

In [ ]:
#Vaja oli kätte saada ka ainult resolutsiooni ja ilma resolutsioonita tulemused

In [ ]:
import numpy as np

In [ ]:
all_embeddings = np.load('BERT_embeddings.npy')

In [ ]:
proovivektoriks1 = np.load('proovivektor.npy')

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
data=pd.read_csv("Koondtabel_lemmadega.csv")

In [ ]:
#test=data[data.Pealkiri1=="Resolutsioon"] #Ainult resolutsiooniga tulemuste saamiseks
test=data[data.Pealkiri1!="Resolutsioon"] #Ilma resolutsioonite tulemuste saamiseks
test

In [ ]:
list_resod=test.index.to_list()
list_resod

In [ ]:
#Loon tabeli, kus on koossiinuse sarnaseimad 200 indeksit, sarnasus ja järjekord (kui sarnane on)
sarnased_maatriks=cosine_similarity(all_embeddings, proovivektoriks1.reshape(1, -1))
#print(sarnased_maatriks)
#print(np.squeeze(sarnased_maatriks))
sorteeritud_indeksid=np.argsort(np.squeeze(sarnased_maatriks))
#print(sorteeritud_indeksid)
abiindeksid=[]
for i in sorteeritud_indeksid:
    if i in list_resod:
        abiindeksid.append(i)
    else:
        i+=1
#print(len(abiindeksid))
listik_indx=[]
for i in range(1,201):
  listik_indx.append(abiindeksid[len(abiindeksid) - i])
koos_dist=sarnased_maatriks[listik_indx]
koos_dist_jarj=[]
for i in koos_dist:
    koos_dist_jarj.append(i[0]) 
#print(koos_dist_jarj)
#print(listik_indx)
tabel_sarnasus = pd.DataFrame(columns=['Kohtuasja_indx', 'Koos_sarnasus', 'Jarjekord'])
tabel_sarnasus['Kohtuasja_indx'] = listik_indx
tabel_sarnasus['Koos_sarnasus'] = koos_dist_jarj
tabel_sarnasus['Jarjekord'] = list(range(1, 201))
Tabel_sarnased=tabel_sarnasus.set_index('Kohtuasja_indx') 
Tabel_sarnased

In [ ]:
filtreeritud_sarnased=data[data.index.isin(listik_indx)]
filtreeritud_sarnased

In [ ]:
# Ühendan kaks tabelit ja filtreerin välja Koos_sarnasus,	Jarjekord, Lõigu sisu, Kohtuasja_nr
Tulemus = pd.concat([Tabel_sarnased, filtreeritud_sarnased], axis=1)
uus = (Tulemus[['Koos_sarnasus', 'Jarjekord','Tag','Lõigu sisu', 'Pealkiri1','Kohtuasja_nr','Kohtuasja_ID', 'algus_lopp', 'Lemma']]).sort_values('Jarjekord', ascending=True)
uus['Indeks'] = uus.index
uus

In [ ]:
#uus.to_csv("bert_tulemus_resod.csv", index=False) #Salvestame ainult resolutsiooniga tulemused
uus.to_csv("bert_tulemus_ilma_resota.csv", index=False) #Salvestame tulemused, milles resolutsioone pole